In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import InputLayer
from keras.layers import Input

import keras
from tensorflow.python.keras.utils.vis_utils import plot_model #useful in visualization of the model
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
nYear=11
simII=150000
param_beta = np.array([-3.5, 2.0, 2.5])

para = pd.Series({'beta': param_beta, 'phi': 0.6, 'sig': 0.8}) # SC1

X = np.stack(( np.repeat(1,simII),
                  np.random.binomial(n=1, p=0.5, size=simII),
                  np.random.binomial(n=1, p=0.5, size=simII)), 
                 axis=1)    
    
X.shape
sig = para['sig']
phi = para['phi']
lamb = np.exp(np.matmul(X, para['beta']))
lamb.shape

n = np.full((simII, nYear), 999)
Rhat = np.full((simII, nYear), 999)
for i in range(simII):
        for t in range(nYear):
        #for t in range((nYear-TT[i]+1),nYear+1):
        #    if t==(nYear-TT[i]+1):
            if t==0 :
                Rhat[i,t] = np.random.normal(0, sig/np.sqrt(1-phi**2), size=1) #= phi*R0[i] + epsilon[i,1]
            else:
                Rhat[i,t] = np.random.normal(Rhat[i,t-1]*phi, sig, size=1) 
                
            n[i,t] = np.random.poisson(lam=lamb[i]*np.exp(Rhat[i,t]), size=1) #rpois(1, mu_n )   


Lamb = lamb[:, np.newaxis]
Lamb = np.concatenate([Lamb]*nYear, axis=1)

N_sim=n
lamb_sim = Lamb

In [ ]:
print(N_sim.shape, lamb_sim.shape)

(150000, 11) (150000, 11)


In [ ]:
pd.DataFrame(N_sim).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,1.082147,1.013740,0.990327,0.987153,0.990380,0.994720,0.994927,0.991373,0.994513,0.992680,1.001680
std,2.549576,2.223672,2.119751,2.115436,2.102271,2.119441,2.106548,2.130677,2.107344,2.079677,2.116415
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,148.000000,76.000000,75.000000,136.000000,68.000000,62.000000,66.000000,64.000000,71.000000,56.000000,69.000000


In [ ]:
pd.DataFrame(lamb_sim).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143,0.836143
std,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617,1.094617
min,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197,0.030197
25%,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130,0.223130
50%,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879,0.367879
75%,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282
max,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282


In [ ]:
print(lamb_sim.shape, N_sim.shape)
XX = np.stack([N_sim[:,:-1], lamb_sim[:,:-1]], axis=2) #shape: [None, 10, 2]
Y = np.stack([N_sim[:,1:], lamb_sim[:,1:]], axis=2)    #shape: [None, 10, 2]

(150000, 11) (150000, 11)


In [ ]:
T=nYear
K=150000
z= 20 #number of BM levels
N_train = int(K*0.7)
N_valid = int(K*0.9)
N_test  = int(K*1.0)
XX_train, Y_train = XX[:N_train, :, :], Y[:N_train,:, :]
XX_valid, Y_valid = XX[N_train:N_valid, :, :], Y[N_train:N_valid, :, :]
XX_test, Y_test = XX[N_valid:, :, :], Y[N_valid:, :, :]
print(XX_train.shape, Y_train.shape, XX_valid.shape, XX_test.shape)


(105000, 10, 2) (105000, 10, 2) (30000, 10, 2) (15000, 10, 2)


In [ ]:
# Code 21-07-25

from tensorflow.keras.layers import Layer
class FinalDense(tf.keras.layers.Layer):

  def __init__(self,units=21, activation=None):
    super(FinalDense, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)
    #D_init = tf.random_normal_initializer()
    #self.D_ = tf.Variable(name="BMS", initial_value=D_init(shape = [5+1], dtype="float32"), trainable=True ) #b0, ..., b20
    
  def build(self, input_shape): #define parameters
    D_init = tf.random_normal_initializer()
    self.D_ = tf.Variable(name="BMS", initial_value=D_init(shape = [5+1], dtype="float32"), trainable=True ) #b0, ..., b20
    #self.D_ = self.add_weight(name="BMS", shape=[5+1], initializer="random_normal", trainable=True)
    super().build(input_shape)

  def call(self, inputs): #inputs: [L(2), .., L(T+1)], [None, T,1] outputs: [R(2), ..., R(T+1)], dim= [None,T,1]
    #tf.nn.embedding_lookup(params=self.D_, ids=L_idx) 
    B=tf.math.cumsum( tf.exp(self.D_) ) #B1<=...<=B20
    R_time = B[0]+ (B[1]-B[0])*(tf.nn.relu(inputs-0.0)-tf.nn.relu(inputs-1.0)) + \
     (B[2]-B[1])*(tf.nn.relu(inputs-1.0)-tf.nn.relu(inputs-2.0)) + \
     (B[3]-B[2])*(tf.nn.relu(inputs-2.0)-tf.nn.relu(inputs-3.0)) + \
     (B[4]-B[3])*(tf.nn.relu(inputs-3.0)-tf.nn.relu(inputs-4.0)) + \
     (B[5]-B[4])*(tf.nn.relu(inputs-4.0)-tf.nn.relu(inputs-5.0)) 
    #Conti_L_time = inputs * self.units  #range = [0,20] #shape = [None,10]
    #L_time = tf.cast(tf.math.maximum(tf.math.minimum(tf.math.ceil(Conti_L_time), self.units), 1), tf.int32)-1
    #L_time : [None, 20] L(2), ..., L(21)
    #R_time = tf.gather_nd(self.R, L_time) #shape = [None,20,1] ? [None,20]?
    return R_time







In [ ]:
def BMS_loss(y_true, y_pred): # y_true [None, T, 2]: t=1~20,     
                                           # y_pred [None, T,1] : R(2) ~ R(T+1)
        lambdas_time = y_true[:,:,1] #shape = [None,T] 
        R_time = y_pred[:,:,0] #shape = [None, T, 1]
        y_hat_time = lambdas_time * R_time #shape = [None,T]
        y_true_time = y_true[:,:,0]  #shape = [None,None]
        mymse = tf.math.reduce_mean((y_true_time-y_hat_time)**2)
        return mymse


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input( shape=(None, 2)) ,
    tf.keras.layers.SimpleRNN(16, return_sequences=True, activation='tanh'),
    tf.keras.layers.SimpleRNN(16, return_sequences=True, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    FinalDense(1)
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 16)          304       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 16)          528       
_________________________________________________________________
dense (Dense)                (None, None, 1)           17        
_________________________________________________________________
final_dense (FinalDense)     (None, None, 1)           6         
Total params: 855
Trainable params: 855
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_input = tf.keras.layers.Input((None, 2)) #input is XX : [None, T, 2] parameters: W
RNN1 = tf.keras.layers.SimpleRNN(16, return_sequences=True, activation='tanh')(rnn_input)
RNN2 = tf.keras.layers.SimpleRNN(16, return_sequences=True, activation='tanh')(RNN1)
rnn_out = tf.keras.layers.Dense(1, activation='sigmoid')(RNN2) #(1/20*L_2, ..., 1/20*L_(T+1)) shape: [None, T, 1] with out=[L(2), ..., L(T+1)]
############################################################################

BMS_out = FinalDense(1)(rnn_out) #yhat = (R_2, ..., R(T+1) )     out [None, T, 1] = [R(2) ,...., R(T+1)] #parametrer: b0, b1, b20

model = keras.Model(inputs=rnn_input, outputs=BMS_out)

In [ ]:
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(FinalDense(1).trainable_variables)

[]


In [ ]:
R=tf.constant([1,2,3,4,5,6,7,8,9,10])
L_time = tf.constant([[[1]],
                      [[3]]])
tf.gather_nd(R, L_time) #shape = [None,20,1] ? [None,20]?

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [4]], dtype=int32)>

In [ ]:
optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss=BMS_loss)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
simple_rnn_10 (SimpleRNN)       (None, None, 16)     304         input_6[0][0]                    
__________________________________________________________________________________________________
simple_rnn_11 (SimpleRNN)       (None, None, 16)     528         simple_rnn_10[0][0]              
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, None, 1)      17          simple_rnn_11[0][0]              
____________________________________________________________________________________________

In [ ]:
model.predict

#######################################################################################

In [ ]:
epochs = 2
batch_size=32
history =model.fit(XX_train, Y_train, batch_size=batch_size, epochs=epochs)#,  validation_data=([XX_valid, MY_ONE], Y_valid)

Epoch 1/2
3282/3282 [==============================] - 15s 4ms/step - loss: 2.6553
Epoch 2/2
3282/3282 [==============================] - 12s 4ms/step - loss: 2.7223


In [ ]:
model.summary()



Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
simple_rnn_8 (SimpleRNN)        (None, None, 16)     304         input_5[0][0]                    
__________________________________________________________________________________________________
simple_rnn_9 (SimpleRNN)        (None, None, 16)     528         simple_rnn_8[0][0]               
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, None, 1)      17          simple_rnn_9[0][0]               
____________________________________________________________________________________________

In [ ]:
inputs = tf.constant([[[1]], 
                      [[2]]])

call2(inputs)

InvalidArgumentError: ignored